# つくレポ数が多いレシピの特徴を抽出する。
まずは個々の料理ではなく、全体的な特徴をみる.

つくレポ数を従属変数、料理名の頻出単語（記号も含む）を説明変数として主成分分析を行い、影響の大きな特徴を抜き出す。

## 料理名の頻出単語と頻出記号を導出する。

In [1]:
from collections import Counter
from typing import List, Iterator, TextIO
import pandas as pd
import numpy as np
import MeCab
import csv
import pprint

上位20位の単語をみる

In [2]:
tagger = MeCab.Tagger('/usr/local/lib/mecab/dic/mecab-ipadic-neologd')
tagger.parse('')

def main(place):
    """
    コマンドライン引数で指定したディレクトリ内のファイルを読み込んで、頻出単語を表示する
    """
    word_list = []

    year = []
    for i in range(1998, 2015):
        year.append(i)

    for i in year:
        df = pd.read_csv(f'/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/recipe/recipe{i}.csv', names=["dish_name"], usecols=[2])
        # 欠損値を補完する
        df["dish_name"].fillna("non-data", inplace=True)

        # データのクレンジングを行う。
        df["dish_name"].replace(" ", "", inplace=True) # 半角スペースを消去する
        df["dish_name"].replace("　", "", inplace=True) # 全角スペースを消去する
        df["dish_name"].replace("〜", "ー", inplace=True) # 伸ばす棒の〜をーに統一する。

        print(f'{i}:')

        frequency = Counter()

        frequency = count_words(df["dish_name"])

        words = []
        for word, count in frequency.most_common(place):
            words.append(word)
            # print(word, count)
        word_list.append(words)
    return word_list

def count_words(data):
    frequency = Counter()

    for content in data:
        words = get_words(content) # 料理名に含まれる名詞のリストを取得する。
        # Counterのupdate()メソッドにリストなどの反復可能オブジェクトを指定すると、
        # リストに含まれる値の出現回数を一度に増やせる
        frequency.update(words)

    print(f'Found {len(frequency)} words from {len(data)} dishes.\n')
    return frequency

def get_words(content):
    """
    文字列内に出現するのリスト（重複含む）を取得する関数
    """

    words = []
    node = tagger.parseToNode(content)
    while node:
        words.append(node.surface)
        node = node.next

    return words

In [3]:
word_list = main(21)

1998:

Found 139 words from 43 dishes.
1999:

Found 702 words from 353 dishes.
2000:

Found 3660 words from 4499 dishes.
2001:

Found 6315 words from 9836 dishes.
2002:

Found 7888 words from 13118 dishes.
2003:

Found 10197 words from 19973 dishes.
2004:

Found 10291 words from 21410 dishes.
2005:

Found 10392 words from 23897 dishes.
2006:

Found 15277 words from 47212 dishes.
2007:

Found 24546 words from 106840 dishes.
2008:

Found 28672 words from 144512 dishes.
2009:

Found 34575 words from 204316 dishes.
2010:

Found 37108 words from 219530 dishes.
2011:

Found 34965 words from 200210 dishes.
2012:

Found 38837 words from 252121 dishes.
2013:

Found 37360 words from 235464 dishes.
2014:

Found 34914 words from 212138 dishes.


In [4]:
# 全ての年で空欄が1位となってしまっているので、それを削除し、csvファイルに書き込んで可視化する。
word_list = np.delete(word_list, 0, 1)

In [5]:
with open("/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/analysis/frequent_words.csv", "a") as f:
    writer = csv.writer(f)
    writer.writerows(word_list)
with open("/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/analysis/frequent_words.csv") as f:
    print(f.read())

'\nwith open("/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/analysis/frequent_words.csv", "a") as f:\n    writer = csv.writer(f)\n    writer.writerows(word_list)\nwith open("/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/analysis/frequent_words.csv") as f:\n    print(f.read())\n'

！や★といった記号が多用されていた。
次は名詞に絞って調べてみる。

In [19]:
def main_noun(place):
    """
    コマンドライン引数で指定したディレクトリ内のファイルを読み込んで、頻出単語を表示する
    """
    word_list_noun = []

    year = []
    for i in range(1998, 2015):
        year.append(i)

    for i in year:
        df = pd.read_csv(f'/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/recipe/recipe{i}.csv', names=["dish_name"], usecols=[2])
        # 欠損値を補完する
        df["dish_name"].fillna("non-data", inplace=True)

        # データのクレンジングを行う。（できてない）
        df["dish_name"].replace(" ", "", inplace=True) # 半角スペースを消去する
        df["dish_name"].replace("　", "", inplace=True) # 全角スペースを消去する
        df["dish_name"].replace("〜", "ー", inplace=True) # 伸ばす棒の〜をーに統一する。

        print(f'{i}:')

        frequency = Counter()

        frequency = count_words_noun(df["dish_name"])

        words = []
        for word, count in frequency.most_common(place):
            words.append(word)
        word_list_noun.append(words)
    return word_list_noun

def count_words_noun(data):
    frequency = Counter()

    for content in data:
        words = get_words_noun(content) # 料理名に含まれる名詞のリストを取得する。
        # Counterのupdate()メソッドにリストなどの反復可能オブジェクトを指定すると、
        # リストに含まれる値の出現回数を一度に増やせる
        frequency.update(words)

    print(f'Found {len(frequency)} nouns from {len(data)} dishes.\n')
    return frequency

def get_words_noun(content):
    """
    文字列内に出現する名詞のリスト（重複含む）を取得する関数
    """
    words = []
    node = tagger.parseToNode(content)
    while node:
        # node.featureはカンマで区切られた文字列なので、split()で分割して
        # 最初の2項目をposとpos_sub1に代入する。posはPart of Speech（品詞）の略
        pos = node.feature.split(',')[0]
        if pos == '名詞':
            words.append(node.surface)
        node = node.next

    return words

In [20]:
word_list_noun = main_noun(20)

1998:
Found 99 nouns from 43 dishes.

1999:
Found 559 nouns from 353 dishes.

2000:
Found 3040 nouns from 4499 dishes.

2001:
Found 5419 nouns from 9836 dishes.

2002:
Found 6784 nouns from 13118 dishes.

2003:
Found 8770 nouns from 19973 dishes.

2004:
Found 8936 nouns from 21410 dishes.

2005:
Found 9039 nouns from 23897 dishes.

2006:
Found 13422 nouns from 47212 dishes.

2007:
Found 21887 nouns from 106840 dishes.

2008:
Found 25673 nouns from 144512 dishes.

2009:
Found 31367 nouns from 204316 dishes.

2010:
Found 33825 nouns from 219530 dishes.

2011:
Found 31699 nouns from 200210 dishes.

2012:
Found 35343 nouns from 252121 dishes.

2013:
Found 33964 nouns from 235464 dishes.

2014:
Found 31524 nouns from 212138 dishes.



In [21]:
with open("/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/analysis/frequent_words.csv", "a") as f:
    writer = csv.writer(f)
    writer.writerows(word_list_noun)
with open("/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/analysis/frequent_words.csv") as f:
    print(f.read())

﻿料理名での頻出単語上位20位,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
の,サラダ,て,、,焼き,！,煮,肉,簡単,リゾット,紅茶,お,に,＆,豆腐,た,で,ホイル,パン,。
の,と,簡単,！,サラダ,風,煮,炒め,揚げ,。,焼き,　,た,で,トマト,な,、,チーズ,お,スープ
の,と,！,簡単,風,サラダ,♪,煮,・,　,焼き,炒め,お,で,☆,スープ,ケーキ,トマト,揚げ,（
の,と,♪,！,☆,簡単,煮,サラダ,風,★,で,炒め,　,・,焼き,お,ケーキ,）,（,豆腐
の,と,！,☆,簡単,♪,サラダ,★,で,　,風,～,煮,お,炒め,・,焼き,）,ケーキ,（
の,と,！,☆,で,♪,簡単,　,サラダ,煮,風,お,★,。,炒め,・,焼き,ケーキ,）,（
の,と,☆,！,簡単,♪,で,　,★,サラダ,お,・,煮,風,炒め,。,ケーキ,）,（,焼き
の,と,☆,！,♪,で,簡単,★,　,サラダ,煮,お,風,）,炒め,ケーキ,（,焼き,・,～
の,☆,と,♪,！,簡単,で,★,サラダ,　,お,煮,炒め,風,焼き,に,ケーキ,豆腐,～,・
の,☆,と,♪,！,簡単,で,★,お,サラダ,　,炒め,に,煮,風,焼き,～,・,豆腐,ケーキ
の,☆,と,♪,簡単,で,！,★,お,　,サラダ,に,炒め,焼き,風,煮,～,✿,♡,・
の,☆,と,簡単,♪,！,で,★,お,　,に,サラダ,炒め,焼き,風,煮,～,野菜,トマト,豆腐
の,☆,と,簡単,♪,で,！,★,お,に,　,炒め,サラダ,風,焼き,煮,＊,チーズ,野菜,豆腐
の,☆,と,簡単,で,♪,！,に,お,★,　,サラダ,炒め,＊,焼き,風,煮,チーズ,野菜,トマト
の,と,☆,簡単,で,！,♪,お,に,サラダ,炒め,★,塩,＊,焼き,　,風,煮,麹,野菜
の,と,簡単,☆,！,で,♪,に,お,♡,サラダ,炒め,焼き,＊,風,★,煮,　,野菜,トマト
の,と,簡単,☆,！,で,♪,♡,に,お,炒め,サラダ,焼き,風,★,煮,＊,トマト,野菜,鶏
,,,,,,,,,,,,,,,,,,,
料理名での頻出名詞上位20位,,,,,,,,,,,,,,,,,,,サラダ,焼き,肉,簡単,リゾット,紅茶,豆腐,ホイル,パン,豚,角,スペシャル,ピーマン,詰め,チーズ,ぎもち,口,あたり,ショコラ・グラ